In [1]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.figure_factory as ff
from ortools.sat.python import cp_model
from bisect import bisect_left, bisect_right
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import unittest
# from src.objects.timeinterval import *
# from src.utils.utils import *
import uuid

In [2]:
intervals = TimeIntervals([TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)),
                           TimeInterval(datetime(2023, 2, 1, 12), datetime(2023, 2, 1, 13)),
                           TimeInterval(datetime(2023, 2, 1, 14), datetime(2023, 2, 1, 16))])
intervals.visualize_gantt()


In [ ]:
# Design classes for Shift scheduling problem (Employee, Shift, Schedule)

# Employee class
# Attributes: name: str, availability: TimeIntervals, assigned shifts: list of Shifts
# Methods: change_availability, change_name

# Shift class
# Attributes: name: str, type: str, interval: TimeInterval, min employees: int, max employees: int, assigned employees: list of Employees
# Methods: assign_employee, unassign_employee, change_interval, change_min_employees, change_max_employees, change_name, change_type

# Single Shift class, which inherits from the Shift class
# Attributes: None
# Methods: None

# Recurring Shift class, which inherits from the Shift class
# Attributes: recurrence interval: timedelta, start date: datetime, end date: datetime, recurrence shifts: list of Shifts
# Methods: get_recurrence_intervals, change_recurrence_interval, change_start_date, change_end_date, get_recurrence_shifts (returns a list of Shifts)

# Schedule class
# Attributes: name: str, employees: list of Employees, shifts: list of Shifts, shift types: list of str, start time: datetime, end time: datetime
# Methods: add_employee, add_shift, reset, change_name, show, change_start_time, change_end_time, solve (solve the shift scheduling problem with OR-Tools and return the solution)



In [15]:
class TimeInterval:
    def __init__(self, start: datetime, end: datetime):
        assert start < end, 'start must be less than end'
        self._start = start
        self._end = end

    @property
    def start(self):
        return self._start

    @start.setter
    def start(self, start: datetime):
        self._start = start

    @property
    def end(self):
        return self._end

    @end.setter
    def end(self, end: datetime):
        self._end = end

    def __repr__(self) -> str:
        return f'TimeInterval({self.start}, {self.end})'

    def __str__(self) -> str:
        return f'{self.start} - {self.end}'

    def __eq__(self, other) -> bool:
        assert isinstance(other, TimeInterval), 'compared object must be of type TimeInterval'
        return self.start == other.start and self.end == other.end

    def overlaps(self, other) -> bool:
        return self.start < other.end and self.end > other.start

    def contains(self, other) -> bool:
        return self.start <= other.start and self.end >= other.end

    def __contains__(self, other) -> bool:
        return self.contains(other)

    def __add__(self, other) -> list:
        """
        Returns a list of TimeIntervals that are the union of self and other
        """
        if self.overlaps(other):
            return [TimeInterval(min(self.start, other.start), max(self.end, other.end))]
        else:
            return [self, other]

    def __sub__(self, other) -> list:
        """
        Returns a list of TimeIntervals that are the difference of self and other
        """
        if self.overlaps(other):
            if self.start < other.start:
                return [TimeInterval(self.start, other.start)]
            elif self.end > other.end:
                return [TimeInterval(other.end, self.end)]
            else:
                return []
        else:
            return [self]



 



class TimeIntervals:
    """
    A class representing a list of time intervals, sorted by start time, and merged if necessary (no overlapping intervals)
    """

    def __init__(self, intervals: list = []):
        assert all([isinstance(interval, TimeInterval) for interval in intervals]), 'All elements in the list must be of type TimeInterval'
        self._intervals = self._merge(intervals)

    def __repr__(self) -> str:
        return f'TimeIntervals({self._intervals})'

    @staticmethod
    def _sort(intervals: list):
        intervals.sort(key=lambda x: x.start)
        return intervals
    
    @staticmethod
    def _merge(intervals: list):
        intervals = TimeIntervals._sort(intervals)
        merged = []
        for interval in intervals:
            if not merged or merged[-1].end < interval.start:
                merged.append(interval)
            else:
                merged[-1].end = max(merged[-1].end, interval.end)
        return merged

    @property
    def intervals(self):
        return self._intervals

    @intervals.setter
    def intervals(self, intervals: list):
        self._intervals = TimeIntervals._merge(intervals)

    def __eq__(self, other) -> bool:
        assert isinstance(other, TimeIntervals), 'compared object must be of type TimeIntervals'
        return self._intervals == other._intervals

    def add(self, interval: TimeInterval):
        self._intervals.append(interval)
        self._intervals = TimeIntervals._merge(self._intervals)

    def remove(self, interval: TimeInterval):
        self._intervals = TimeIntervals._subtract(self._intervals, interval)

    @staticmethod
    def _subtract(intervals: list, interval: TimeInterval):
        intervals = TimeIntervals._sort(intervals)
        subtracted = []
        for i in intervals:
            if i.overlaps(interval):
                if i.start < interval.start:
                    subtracted.append(TimeInterval(i.start, interval.start))
                if i.end > interval.end:
                    subtracted.append(TimeInterval(interval.end, i.end))
            else:
                subtracted.append(i)
        return subtracted

        

        

        


        


    


    

In [94]:
# Test the TimeInterval classes
# 1. test initialization
# 2. test overlaps
# 3. test contains
# 4. test subtraction
# 5. test addition

class TestTimeInterval(unittest.TestCase):
    dt1 = datetime(2023, 2, 1, 10)
    dt2 = datetime(2023, 2, 1, 11)
    dt3 = datetime(2023, 2, 1, 12)
    dt4 = datetime(2023, 2, 1, 13)

    ti1 = TimeInterval(dt1, dt2)
    ti2 = TimeInterval(dt2, dt3)
    ti3 = TimeInterval(dt3, dt4)
    ti4 = TimeInterval(dt1, dt4)
    ti5 = TimeInterval(dt1, dt3)
    ti6 = TimeInterval(dt2, dt4)


    def test_init(self):
        self.assertEqual(self.ti1.start, self.dt1)
        self.assertEqual(self.ti1.end, self.dt2)
    
    def test_overlaps(self):
        self.assertTrue(self.ti1.overlaps(self.ti4))
        self.assertTrue(self.ti4.overlaps(self.ti1))
        self.assertTrue(self.ti5.overlaps(self.ti6))
        self.assertTrue(self.ti6.overlaps(self.ti5))
        self.assertFalse(self.ti1.overlaps(self.ti2))
        self.assertFalse(self.ti2.overlaps(self.ti1))
        self.assertFalse(self.ti1.overlaps(self.ti3))
        self.assertFalse(self.ti3.overlaps(self.ti1))


    def test_contains(self):
        self.assertTrue(self.ti4.contains(self.ti1))
        self.assertTrue(self.ti4.contains(self.ti2))
        self.assertFalse(self.ti1.contains(self.ti4))
        self.assertFalse(self.ti2.contains(self.ti4))
        self.assertFalse(self.ti1.contains(self.ti2))
        self.assertFalse(self.ti2.contains(self.ti1))
        self.assertTrue(self.ti1.contains(self.ti1))

    def test_subtraction(self):
        self.assertEqual(self.ti1 - self.ti2, [self.ti1])
        self.assertEqual(self.ti2 - self.ti1, [self.ti2])
        self.assertEqual(self.ti1 - self.ti4, [])
        self.assertEqual(self.ti4 - self.ti1, [self.ti6])

unittest.main(argv=[''], verbosity=2, exit=False)



test_contains (__main__.TestTimeInterval) ... ok
test_init (__main__.TestTimeInterval) ... ok
test_overlaps (__main__.TestTimeInterval) ... ok
test_subtraction (__main__.TestTimeInterval) ... ok
test_add (__main__.TestTimeIntervals) ... FAIL
test_eq (__main__.TestTimeIntervals) ... ok
test_init (__main__.TestTimeIntervals) ... ok
test_remove (__main__.TestTimeIntervals) ... ok

FAIL: test_add (__main__.TestTimeIntervals)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/qm/hc980d_116zfqwx_9dxh3dmm0000gn/T/ipykernel_54317/289041481.py", line 17, in test_add
    self.assertEqual(intervals.intervals, [TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 13))])
AssertionError: Lists differ: [TimeInterval(2023-02-01 10:00:00, 2023-02-01 14:00:00)] != [TimeInterval(2023-02-01 10:00:00, 2023-02-01 13:00:00)]

First differing element 0:
TimeInterval(2023-02-01 10:00:00, 2023-02-01 14:00:00)
TimeInterval(2023-02

In [9]:
# Test the TimeIntervals class
# 1. test initialization
# 2. test add
# 3. test remove
class TestTimeIntervals(unittest.TestCase):
    iv1 = TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 12))
    iv2 = TimeInterval(datetime(2023, 2, 1, 11), datetime(2023, 2, 1, 13))
    iv3 = TimeInterval(datetime(2023, 2, 1, 12), datetime(2023, 2, 1, 14))

    def test_init(self):
        intervals = TimeIntervals([self.iv1, self.iv2, self.iv3])
        self.assertEqual(intervals.intervals, [TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 14))])

    def test_add(self):
        intervals = TimeIntervals([self.iv1])
        intervals.add(self.iv2)
        self.assertEqual(intervals.intervals, [TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 13))])

    def test_remove(self):
        intervals = TimeIntervals([self.iv1, self.iv2, self.iv3])
        intervals.remove(self.iv2)
        self.assertEqual(intervals.intervals, [TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)), TimeInterval(datetime(2023, 2, 1, 13), datetime(2023, 2, 1, 14))])
        
    def test_eq(self):
        intervals1 = TimeIntervals([self.iv1, self.iv2, self.iv3])
        intervals2 = TimeIntervals([self.iv1, self.iv3])
        self.assertTrue(intervals1 == intervals2)
        intervals1.remove(self.iv3)
        self.assertFalse(intervals1 == intervals2)

unittest.main(argv=[''], verbosity=2, exit=False)




test_add (__main__.TestTimeIntervals) ... ok
test_eq (__main__.TestTimeIntervals) ... ok
test_init (__main__.TestTimeIntervals) ... ok
test_remove (__main__.TestTimeIntervals) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


In [16]:
# Employee class
class Employee:
    def __init__(self, name: str = "New Employee", role: str = "Uncategorized"):
        self._name = name
        self.role = role
        self._availability = TimeIntervals() 
        self.assigned_shifts = []

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name: str):
        self._name = name

    @property
    def availability(self):
        return self._availability

    @availability.setter
    def availability(self, availability: TimeIntervals):
        self._availability = availability

    @property
    def role(self):
        return self.role
    
    @role.setter
    def role(self, role: str):
        self.role = role
    
    def _add_availability(self, interval: TimeInterval):
        self._availability.add(interval)

    def _remove_availability(self, interval: TimeInterval):
        self._availability.remove(interval)



# Shift class
class Shift:
    def __init__(self, interval: TimeInterval, min_employees: int, max_employees: int, name: str = "New shift", type: str = "Uncategorized"):
        assert min_employees <= max_employees, "Minimum number of employees must be less than or equal to the maximum number of employees"
        self._name = name
        self._type = type
        self._interval = interval
        self._min_employees = min_employees
        self._max_employees = max_employees
        self._assigned_employees = []

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name: str):
        self._name = name

    @property
    def type(self):
        return self._type
    
    @type.setter
    def type(self, type: str):
        self._type = type
    
    @property
    def interval(self):
        return self._interval
    
    @interval.setter
    def interval(self, interval: TimeInterval):
        self._interval = interval

    @property
    def min_employees(self):
        return self._min_employees

    @min_employees.setter
    def min_employees(self, min_employees: int):
        self._min_employees = min_employees

    @property
    def max_employees(self):
        return self._max_employees

    @max_employees.setter
    def max_employees(self, max_employees: int):
        self._max_employees = max_employees

    @property
    def date(self):
        return self.interval.start.date()

    @property
    def duration(self):
        return self.interval.end - self.interval.start

    def _add_employee(self, employee: Employee):
        self._assigned_employees.append(employee)



# Schedule class
class Schedule:
    def __init__(self, start_date: datetime, end_date: datetime, name: str = "New schedule"):
        self._name = name
        self._shifts = []
        self._employees = []
        self._start_date = start_date
        self._end_date = end_date

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name: str):
        self._name = name   

    @property
    def shifts(self):
        return self._shifts
    
    @shifts.setter
    def shifts(self, shifts: list):
        self._shifts = shifts

    @property
    def employees(self):
        return self._employees

    @employees.setter
    def employees(self, employees: list):
        self._employees = employees

    @property
    def start_date(self):
        return self._start_date

    @start_date.setter
    def start_date(self, start_date: datetime):
        self._start_date = start_date

    @property
    def end_date(self):
        return self._end_date

    @end_date.setter
    def end_date(self, end_date: datetime):
        self._end_date = end_date

    


    

In [ ]:
# Pseudo code for the scheduling algorithm

# schedule = Schedule(start_date, end_date)
# # recurring is set to False by default, recurring_interval is set to 1 day by default
# shift1 = schedule.create_shift(name, type, interval, min_employees, max_employees, recurring = False, recurring_interval) 
# # set recurring to True to create a recurring shift with a recurring interval of 1 day until the end of the schedule
# shifts = schedule.create_shift(name, type, interval, min_employees, max_employees, recurring = True, recurring_interval = timedelta(days = 1)) 
# # remove some shifts from the schedule with filter
# for shift in shifts:
#     if shift.:

# employee = schedule.create_employees(name, role) # availability is set to the entire schedule by default
# employee.availability.remove(interval) # remove availability from employee 
# schedule.assign_employees(shift, employee) # manually assign employee to shift, if employee is not available for the shift, an error is raised

# # generate schedule based on the constraints of the shifts and employees and return a list of shifts with assigned employees (CP-SAT)
# schedule.generate_schedule() 

# # display the schedule
# schedule.display()

# # export the schedule to a file, to csv by default
# schedule.save()

# # load the schedule from a file
# schedule.load()

